In [1]:
%%capture --no-stdout
%reload_ext watermark
%watermark -uniz --author "Prayson W. Daniel" -vm -p duckdb,polars,weaviate,ollama

Author: Prayson W. Daniel

Last updated: 2024-11-19T16:38:19.928892+01:00

Python implementation: CPython
Python version       : 3.11.10
IPython version      : 8.29.0

duckdb  : 1.1.3
polars  : 1.13.0
weaviate: 4.9.3
ollama  : 0.3.3

Compiler    : Clang 15.0.0 (clang-1500.3.9.4)
OS          : Darwin
Release     : 23.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit



In [2]:
import weaviate
from weaviate.classes.config import Configure

In [3]:
client = weaviate.connect_to_local()
client.collections.delete_all()

In [4]:
COLLECTION_NAME: str = "DataX"
client.collections.create(
    COLLECTION_NAME,
    vectorizer_config=[
        Configure.NamedVectors.text2vec_ollama(
            name="ds_search",
            source_properties=["description"],
            api_endpoint="http://host.docker.internal:11434",
            model="nomic-embed-text",
        )
    ],
    generative_config=Configure.Generative.ollama(
        api_endpoint="http://host.docker.internal:11434",
        model="llama3.2",
    ),
)

In [5]:
objects = [
    {
        "title": "Shahnoza",
        "description": "Senior Data Scientist & Tech Lead | Machine Learning & MLOps",
    },
    {
        "title": "Mette",
        "description": "Data Scientist | AI & Machine Learning Engineer | Physicist",
    },
    {
        "title": "Ulrik",
        "description": "Senior Data Scientist @NTT Data / Machine Learning & MLOps / Biophysicist turned Data Scientist",
    },
    {
        "title": "Manuel",
        "description": "Data Scientist & AI Engineer @ NTT Data | MSc Computer Science",
    },
    {
        "title": "Nicolai",
        "description": "Lead AI Solution Architect @ NTT DATA | Teaching AI & ML @ CBS",
    },
]

In [6]:
collection = client.collections.get(COLLECTION_NAME)

with collection.batch.dynamic() as batch:
    for obj in objects:
        weaviate_obj = {
            "title": obj["title"],
            "description": obj["description"],
        }

        batch.add_object(
            properties=weaviate_obj,
        )

In [7]:
query = "Who are MLOps?"
response = collection.query.hybrid(query=query, limit=3, alpha=1)

In [8]:
for obj in response.objects:
    print(f'{obj.properties["title"]} - {obj.properties["description"]}')

Shahnoza - Senior Data Scientist & Tech Lead | Machine Learning & MLOps
Ulrik - Senior Data Scientist @NTT Data / Machine Learning & MLOps / Biophysicist turned Data Scientist
Mette - Data Scientist | AI & Machine Learning Engineer | Physicist


In [15]:
response = collection.generate.hybrid(
    query=query,
    limit=3,
    single_prompt="""
        You are a story teller: Tell a two paragraphs story about {title}:
    """,
)

In [16]:
for obj in response.objects:
    print(f"{'=':=>120}\n")
    print(f"{obj.properties['title']}")
    print(f"{obj.generated}")


Shahnoza
In the heart of ancient Samarkand, there lived a young girl named Shahnoza. She was known for her extraordinary beauty and kindness, but what truly set her apart were the mysteries that shrouded her past. It was said that Shahnoza possessed magical powers, inherited from her ancestors who had been revered as healers and wise women in their time. As she grew older, Shahnoza's powers only intensified, allowing her to communicate with animals and sense the deepest desires of those around her. Despite the whispers of prophecy surrounding her, Shahnoza remained humble and dedicated to using her gifts to help others.

One fateful day, a devastating drought struck the land, threatening the livelihoods of the people of Samarkand. Crops began to wither, rivers dried up, and the once-thriving city teetered on the brink of collapse. Shahnoza, sensing the desperation that hung in the air, knew she had to act. With her eyes closed, she focused all her energy on finding a solution to the d